# Pre-processing External Datasets

In [24]:
import pandas as pd
import re
import numpy as np
from ast import literal_eval

In [2]:
# Read in datasets
income_df = pd.read_csv("datasource-AU_Govt_ABS-UoM_AURIN_DB_3_abs_personal_income_total_income_sa2_2011_2018.csv")
population_df = pd.read_csv("datasource-AU_Govt_ABS-UoM_AURIN_DB_3_abs_regional_population_sa2_2001_2021.csv")

# Drop irrelevant attributes for income dataset
suburb_info = income_df[["sa2_code", "sa2_name"]]
income_df = income_df.loc[:, "mean_aud_2011_12":]
income_df = income_df.join(suburb_info)

# Rename attributes for income dataset
income_df.columns = ["mean_income_2011_12", "mean_income_2012_13", "mean_income_2013_14", "mean_income_2014_15", 
                     "mean_income_2015_16", "mean_income_2016_17", "mean_income_2017_18", "geometries", "suburb_code", 
                     "suburb"]

# Drop irrelevant attributes for population dataset
estimated_population = population_df.loc[:, "erp_2001":"erp_2021"]
population_df = population_df[["state_name_2016", "sa2_maincode_2016", "sa2_name_2016", "erp_change_per_cent_2020_21", 
                               "area_km2", "pop_density_2021_people_per_km2", "births_2016_17", "deaths_2016_17", 
                               "births_2017_18", "deaths_2017_18", "births_2018_19", "deaths_2018_19", "births_2019_20", 
                               "deaths_2019_20", "births_2020_21", "deaths_2020_21", "geom"]]
population_df = population_df.join(estimated_population)

# Rename attributes for population dataset
population_df.rename(columns={"state_name_2016":"state", "sa2_maincode_2016": "suburb_code", "sa2_name_2016": 
                                              "suburb", "erp_change_per_cent_2020_21": "change_in_population_percentage", 
                                              "area_km2": "suburb_area_km2", "pop_density_2021_people_per_km2": 
                                              "population_density_per_km2", "erp_2001": "population_2001", 
                                              "erp_2002": "population_2002", "erp_2003": "population_2003", 
                                              "erp_2004": "population_2004", "erp_2005": "population_2005", 
                                              "erp_2006": "population_2006", "erp_2007": "population_2007", 
                                              "erp_2008": "population_2008", "erp_2009": "population_2009", 
                                              "erp_2010": "population_2010", "erp_2011": "population_2011", 
                                              "erp_2012": "population_2012", "erp_2013": "population_2013", 
                                              "erp_2014": "population_2014", "erp_2015": "population_2015", 
                                              "erp_2016": "population_2016", "erp_2017": "population_2017", 
                                              "erp_2018": "population_2018", "erp_2019": "population_2019", 
                                              "erp_2020": "population_2020", "erp_2021": "population_2021"}, inplace=True)

# Filter instances in Victoria state for population dataset
population_df = population_df[population_df["state"] == "Victoria"]
suburb_code = population_df[["suburb_code"]]

# Filter instances in Victoria state for income dataset
income_df = pd.merge(income_df, suburb_code, on="suburb_code")

# Drop rows containing at least one null value in income dataset
income_df = income_df.dropna(axis=0, how ="any") 

# Drop rows containing at least one null value in popoulation dataset
population_df = population_df.dropna(axis=0, how ="any") 

# Save dataframes as csv files
income_df.to_csv("income.csv", header=False, index=False)
population_df.to_csv("population.csv", header=False, index=False)

# Feature Engineering Primary Dataset

In [73]:
# Read in primary dataset
rental_df = pd.read_csv("All_Houses_Scraped.csv")

# Obtain all suburb names in Victoria
state_suburbs_df = pd.read_csv("georef-australia-state-suburb.csv")
state_suburbs_df = state_suburbs_df[state_suburbs_df["Official Name State"] == "Victoria"]
suburbs = [suburb.lower() for suburb in state_suburbs_df["Official Name Suburb"]]
for i in range(len(suburbs)):
    suburbs[i] = re.sub(r'\(.*\)', '', suburbs[i])
    suburbs[i] = suburbs[i].strip()

extra_suburbs = ["preston west", "prahran east", "sanctuary lakes", "mallacoota"]
for i in range(len(extra_suburbs)):
    suburbs.append(extra_suburbs[i])
    
# Convert address to list
rental_df["name"] = rental_df["name"].str.split(' ')

# Create new columns for potential suburb names
rental_df["name_one"] = [row[-3].lower() for row in rental_df["name"]]
rental_df["name_two"] = [row[-4:-2] for row in rental_df["name"]]
rental_df["name_three"] = [row[-5:-2] for row in rental_df["name"]]

# Convert potential suburb names from list to string
rental_df["name_two"] = rental_df["name_two"].str.join(' ')
rental_df["name_two"] = [row.lower() for row in rental_df["name_two"]]
rental_df["name_three"] = rental_df["name_three"].str.join(' ')
rental_df["name_three"] = [row.lower() for row in rental_df["name_three"]]

i = 0
for suburb_one, suburb_two, suburb_three in zip(rental_df.iloc[:, -3], rental_df.iloc[:, -2], rental_df.iloc[:, -1]):
    if suburb_three in suburbs:
        rental_df.loc[i, ["suburb"]] = suburb_three
    elif suburb_two in suburbs:
        rental_df.loc[i, ["suburb"]] = suburb_two
    elif suburb_one in suburbs:
        rental_df.loc[i, ["suburb"]] = suburb_one
    i += 1

rental_df = rental_df.drop(columns=["name_one", "name_two", "name_three"])

# Drop rows containing at least one null value in primary dataset
rental_df = rental_df.dropna(axis=0, how ="any") 

In [74]:
# Drop rows without room information
rental_df["rooms"] = rental_df["rooms"].replace("[]", np.nan)
rental_df = rental_df.dropna(subset = ["rooms"] , how="any").reset_index(drop=True)

# Set up cases for extracting room information
info_one = '|'.join(["bed"])
info_two = '|'.join(["bath"])
info_three = '|'.join(["park"])
numbers = '\d+[.,]?\d*'                              
case_one = fr'({numbers})(?:[\s\d\-\+\/]*)(?:{info_one})'
case_two = fr'({numbers})(?:[\s\d\-\+\/]*)(?:{info_two})'
case_three = fr'({numbers})(?:[\s\d\-\+\/]*)(?:{info_three})'
pattern_one = re.compile(case_one) 
pattern_two = re.compile(case_two) 
pattern_three = re.compile(case_three) 

# Create columns for the number of beedrooms, bathrooms and parking spaces
for i in range(len(rental_df)):
    rental_df["rooms"].loc[i] = literal_eval(rental_df["rooms"].loc[i])
    for j in range(len(rental_df["rooms"].loc[i])):
        rental_df["rooms"].loc[i][j] = rental_df["rooms"].loc[i][j].lower()     
        if pattern_one.findall(rental_df["rooms"].loc[i][j]) != []:
            rental_df.loc[i, "bedrooms"] = pattern_one.findall(rental_df["rooms"].loc[i][j])
        elif pattern_two.findall(rental_df["rooms"].loc[i][j]) != []:
            rental_df.loc[i, "bathrooms"] = pattern_two.findall(rental_df["rooms"].loc[i][j])
        else:
            rental_df.loc[i, "parking_spaces"] = pattern_three.findall(rental_df["rooms"].loc[i][j])

# Preliminary Analysis

In [75]:
print(rental_df.head(30))

                                           Unnamed: 0  \
0   https://www.domain.com.au/15-fintonia-street-h...   
1   https://www.domain.com.au/41-bishop-street-oak...   
2   https://www.domain.com.au/109-6-dalgety-street...   
3   https://www.domain.com.au/16-30-swindon-road-o...   
4   https://www.domain.com.au/1-14-bletchley-road-...   
5   https://www.domain.com.au/26-austin-street-hug...   
6   https://www.domain.com.au/2-67-euston-road-hug...   
7   https://www.domain.com.au/5-158-kangaroo-road-...   
8   https://www.domain.com.au/3-13-state-street-oa...   
9   https://www.domain.com.au/24-euston-road-hughe...   
10  https://www.domain.com.au/1-61-macrina-street-...   
11  https://www.domain.com.au/2a-grant-street-oakl...   
12  https://www.domain.com.au/2-52-william-street-...   
13  https://www.domain.com.au/1652-dandenong-road-...   
14  https://www.domain.com.au/4-13-elizabeth-stree...   
15  https://www.domain.com.au/406-89-atherton-road...   
16  https://www.domain.com.au/1